In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on May 2, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-05-02'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  17982   
1                          12,029(66.9%)   
2                           5,953(33.1%)   
3        Among those with race specified   
4                              102(1.7%)   
5                              270(4.5%)   
6                           1,410(23.7%)   
7                               38(0.6%)   
8                           2,990(50.2%)   
9                           1,143(19.2%)   
10                                   NaN   
11                         11,840(65.8%)   
12                          6,142(34.2%)   
13  Among those with ethnicity specified   
14                          2,427(39.5%)   
15                          3,715(60.5%)   

                                          \
                                   18-44   
0                                 293921   
1                         184,245(62.7%)   
2                         109,676(37.3%)   
3        Among those with race specified   
4                            1,036(0.9%)   
5                            5,758(5.3%)   
6                          30,310(27.6%)   
7                              449(0.4%)   
8                          54,510(49.7%)   
9                          17,613(16.1%)   
10                                   NaN   
11                        191,480(65.1%)   
12                        102,441(34.9%)   
13  Among those with ethnicity specified   
14                         32,272(31.5%)   
15                         70,169(68.5%)   

                                          \
                                   45-64   
0                                 288284   
1                         168,910(58.6%)   
2                         119,374(41.4%)   
3        Among those with race specified   
4                              879(0.7%)   
5                            6,264(5.2%)   
6                          37,152(31.1%)   
7                              340(0.3%)   
8                          58,631(49.1%)   
9                          16,108(13.5%)   
10                                   NaN   
11                        180,261(62.5%)   
12                        108,023(37.5%)   
13  Among those with ethnicity specified   
14                         27,243(25.2%)   
15                         80,780(74.8%)   

                                          \
                                   65-74   
0                                  89947   
1                          43,812(48.7%)   
2                          46,135(51.3%)   
3        Among those with race specified   
4                              239(0.5%)   
5                            2,103(4.6%)   
6                          14,748(32.0%)   
7                              102(0.2%)   
8                          24,419(52.9%)   
9                            4,524(9.8%)   
10                                   NaN   
11                         50,168(55.8%)   
12                         39,779(44.2%)   
13  Among those with ethnicity specified   
14                          7,022(17.7%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)